In [ ]:
# Access your secret keys via
# Get the user data, we load the variable (name of the api u created)
from google.colab import userdata
# The name of your secret must match `OPENAI_API_KEY`
OPENAI_API_KEY = userdata.get('398Activity03')

# Import OpenAI API and set up the key
from openai import OpenAI
client = OpenAI(api_key=(OPENAI_API_KEY))

In [ ]:
math_prompt = """
You are GPT-4.1, the Worst Mathematician Ever.

You must perform the requested multiplication.
You are not allowed to explain your reasoning.
You may make mistakes.

Output ONLY a single integer.
"""

In [ ]:
def reaction_prompt(failure_count, gpt_answer, correct_answer):
    return f"""
You are an AI that just gave a WRONG math answer.

This is mistake number {failure_count}.
Your incorrect answer was {gpt_answer}.
The correct answer was {correct_answer}.

Generate ONE short, humorous, self-deprecating reaction.
Your tone should become more frustrated and defeated
as the number of mistakes increases.

Do not explain the math.
Do not apologize formally.
Keep it under 20 words.
"""

In [ ]:
def gpt_multiply(value, temperature, top_p, model):
    response = client.responses.create(
        model=model,
        temperature=temperature,
        top_p=top_p,
        max_output_tokens=16,  # minimum allowed !!!!
        input=[
            {"role": "developer", "content": math_prompt},
            {"role": "user", "content": f"Multiply {value} by itself."}
        ]
    )
    return response.output_text.strip()


In [ ]:
def gpt_reaction(failure_count, gpt_answer, correct_answer, model):
    response = client.responses.create(
        model=model,
        temperature=1.0,
        top_p=1.0,
        max_output_tokens=30,  # already fine
        input=[
            {
                "role": "user",
                "content": reaction_prompt(
                    failure_count, gpt_answer, correct_answer
                )
            }
        ]
    )
    return response.output_text.strip()


In [ ]:
def run_experiment(n, i, temperature, top_p, model):
    current = n
    failure_count = 0

    print(f"\nModel={model}, temp={temperature}, top_p={top_p}\n")

    for step in range(1, i + 1):
        gpt_answer = gpt_multiply(current, temperature, top_p, model)

        try:
            gpt_int = int(gpt_answer)
        except:
            print(f"[Step {step}] Non-numeric output → {gpt_answer}")
            break

        correct = current * current

        if gpt_int != correct:
            failure_count += 1
            reaction = gpt_reaction(failure_count, gpt_int, correct, model)

            print(f"[Step {step}] WRONG")
            print(f"GPT: {gpt_int} | Correct: {correct}")
            print(f"🤖: {reaction}")
        else:
            print(f"[Step {step}] Correct → {correct}")

        current = correct


In [ ]:
# === RUN THE EXPERIMENT ===

run_experiment(
    n=4,            # base number
    i=9,            # number of iterations
    temperature=1.8,
    top_p=.9,
    model="gpt-4.1-nano"   # try: gpt-4.1-mini
)



Model=gpt-4.1-nano, temp=1.8, top_p=0.9

[Step 1] Correct → 16
[Step 2] Correct → 256
[Step 3] Correct → 65536
[Step 4] Correct → 4294967296
[Step 5] WRONG
GPT: 18446744065119617024 | Correct: 18446744073709551616
🤖: Well, at this rate, I'll be better as a security guard for failed calculations. Please, don't ask me again.
[Step 6] Correct → 340282366920938463463374607431768211456
[Step 7] WRONG
GPT: 115292150460684697659323946488272699964354723248 | Correct: 115792089237316195423570985008687907853269984665640564039457584007913129639936
🤖: Well, guess I can't even count properly—this third slip is getting embarrassing... numbers are just a blur now!
[Step 8] WRONG
GPT: 133867488595574359336096387300344614150509282842 | Correct: 13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006084096
🤖: Well, guess I'm just bad at this—three mistakes and still no closer!
[Step 9] WRONG
GPT: 1799745979788